In [1]:
import numpy as np
import pandas as pd
import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import preprocessingWithMissingvalues

In [2]:
carDf= pd.read_csv('oneHotPreprocess.csv')
carDf.head()

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,Deductible,DriverRating,DaysDiff,Lable
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0.2625,0.000000,0.000000,0.071795,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.4250,1.000000,1.000000,0.069231,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.5875,0.666667,0.666667,0.089744,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.8125,0.333333,0.333333,0.107692,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.3375,0.000000,0.000000,0.094872,0


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;

In [4]:
carDateNormalized= carDf

In [5]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])

print("nonFraudulent:",type(nonFraudulent))
print("carDateNormalized:",type(carDateNormalized))

j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1


nonFraudulent: <class 'pandas.core.frame.DataFrame'>
carDateNormalized: <class 'pandas.core.frame.DataFrame'>


In [6]:
print(type(nonFraudulent))
print(type(fraudulent))

#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [7]:
#split the dataset into train and test set in equal proportion
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.3)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulent, fraudulentLable,random_state=3,test_size=0.3)


In [8]:
#combine the fraudulent and nonfraudulent training and test set to make the newTrain and newTest

#create the dataframe for fraudulent and non-fraudulent data
newCombineTrain= pd.DataFrame(columns=columnlable)
newCombineTrainLable= pd.DataFrame(columns=['lable'])

newCombineTest= pd.DataFrame(columns=columnlable)
newCombineTestLable= pd.DataFrame(columns=['lable'])



#combine trainset
framesFeatures= [nonFraudX_train,fraudX_train]
framesLable= [nonFraudY_train,fraudY_train]
newCombineTrain= pd.concat(framesFeatures)
newCombineTrainLable= pd.concat(framesLable)


framesFeatures2= [nonFraudX_test,fraudX_test]
framesLable2= [nonFraudY_test,fraudY_test]
newCombineTest= pd.concat(framesFeatures2)
newCombineTestLable= pd.concat(framesLable2)

#make the dataset as values
#newCombineTrain= newCombineTrain.values
#newCombineTrainLable= newCombineTrainLable.values


#divide the trainset into trainset and validation set
newX_train,newX_trainV,newY_train,newY_trainV= train_test_split(newCombineTrain,newCombineTrainLable,
                                                                random_state=3,test_size=0.2)


newCombineTest= newCombineTest.values
newCombineTestLable= newCombineTestLable.values

newCombineTrain= newX_train.values
newCombineTrainLable= newY_train.values

validX_train= newX_trainV.values
validY_test= newY_trainV.values
print("TrainData set size:",newCombineTrain.shape[0])
print("Test Data set size:",newCombineTest.shape[0])
print("ValidData set size:",validX_train.shape[0])

TrainData set size: 8634
Test Data set size: 4626
ValidData set size: 2159


In [9]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [10]:
#make the for train and test loader
myTrainDataset= myDataset(newCombineTrain,newCombineTrainLable)
myTestDataset= myDataset(newCombineTest,newCombineTestLable)
myValidDataset= myDataset(validX_train,validY_test)

In [21]:
#make the trainloader and test loader for nonfraudulent dataset.
trainLoader= torch.utils.data.DataLoader(myTrainDataset,batch_size=32,shuffle=True,num_workers=0)
testLoader= torch.utils.data.DataLoader(myTestDataset,batch_size=1,shuffle=True,num_workers=0)
validLoader= torch.utils.data.DataLoader(myValidDataset,batch_size=32,shuffle=True,num_workers=0)

newTrainLoader= torch.utils.data.DataLoader(myTrainDataset,batch_size=1,shuffle=True,num_workers=0)



In [12]:
#Network architecture for the base autoencoders
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.fc1= nn.Linear(85,64)
        self.act1= nn.LeakyReLU()
        self.fc2= nn.Linear(64,43)
        self.act2= nn.LeakyReLU()
        self.fc3= nn.Linear(43,24)
        self.act3= nn.LeakyReLU()
        self.fc4= nn.Linear(24,10)
        self.act4= nn.LeakyReLU()
        self.fc5= nn.Linear(10,2)
        self.act5= nn.Sigmoid();
            
    def forward(self,x):
        x= self.fc1(x)
        x= self.act1(x)
        x= self.fc2(x)
        x= self.act2(x)
        x= self.fc3(x)
        x= self.act3(x)
        x= self.fc4(x)
        x= self.act4(x)
        x= self.fc5(x)
        x= self.act5(x)
        return x;

In [13]:
model= Network()
#criterion= nn.CrossEntropyLoss()
criterion= nn.BCELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.0001,momentum=0.9,weight_decay=0.0001)

In [14]:
#def validation Test
def validTest(trainedModel,loader):
    running_loss= 0.0
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= trainedModel(inputs)
        lables= torch.tensor(lable).float()
        
        newMaxProb,maxIndex = torch.topk(output,1,dim=1)
        loss= criterion(newMaxProb,lables.squeeze())
        
        running_loss += loss.item()
    
    return running_loss/validX_train.shape[0]

In [15]:
#train the model
training_loss= []
validation_loss= []
for epoch in range(500):
    running_loss= 0.0
    for i, (feature,lable) in enumerate(trainLoader):
        
        #gets the inputs
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable).float()
        #lables= lable.type(torch.LongTensor)
        #print("lables:",lables)
        #print("lables.size:",lables.size())
       
        # =====================forward====================
        output = model(inputs)
        
        #print("output:",output)
        #print("output.size:",output.size())
        
        #print("output.requires_grad:",output.requires_grad)
        
        """
        maxIndex= torch.argmax(output,dim=1)
        maxProb= []
        tensorLength= output.size(0)
        print("maxIndex:",maxIndex)
        
        for i in range(tensorLength):
            index= maxIndex[i]
            maxProb.append(output[i][index].item())
        newMaxProb= torch.tensor(maxProb)
        newMaxProb= newMaxProb.view(-1,1)
        """
        
        newMaxProb,_ = torch.topk(output,1,dim=1)
        #print("newMaxProb:",newMaxProb)
        #print("newMaxProb.size:",newMaxProb.size())
        #print("newMaxProb.requires_grad:",newMaxProb.requires_grad)
                        
        #loss= criterion(output,lables.squeeze())
        #output= newMaxProb
        loss= criterion(newMaxProb,lables.squeeze())
        
        #print("loss:",loss)
        
        # ===================backward====================
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # =======print the statistics
        running_loss += loss.item()
        
    print('[%d] loss: %.6f' %(epoch + 1,  running_loss /newCombineTrain.shape[0]))
    validLoss= validTest(model,validLoader)
    validation_loss.append(validLoss)
    
    training_loss.append(running_loss/newCombineTrain.shape[0])
    
    

/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/usr/local/anaconda35/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[1] loss: 0.026270
[2] loss: 0.024704
[3] loss: 0.023634
[4] loss: 0.022674
[5] loss: 0.021777
[6] loss: 0.020937
[7] loss: 0.020150
[8] loss: 0.019410
[9] loss: 0.018714
[10] loss: 0.018059
[11] loss: 0.017439
[12] loss: 0.016855
[13] loss: 0.016301
[14] loss: 0.015776
[15] loss: 0.015278
[16] loss: 0.014802
[17] loss: 0.014350
[18] loss: 0.013918
[19] loss: 0.013504
[20] loss: 0.013110
[21] loss: 0.012731
[22] loss: 0.012368
[23] loss: 0.012019
[24] loss: 0.011684
[25] loss: 0.011363
[26] loss: 0.011052
[27] loss: 0.010753
[28] loss: 0.010467
[29] loss: 0.010192
[30] loss: 0.009932
[31] loss: 0.009679
[32] loss: 0.009438
[33] loss: 0.009211
[34] loss: 0.008999
[35] loss: 0.008794
[36] loss: 0.008604
[37] loss: 0.008423
[38] loss: 0.008262
[39] loss: 0.008109
[40] loss: 0.007969
[41] loss: 0.007849
[42] loss: 0.007739
[43] loss: 0.007633
[44] loss: 0.007546
[45] loss: 0.007471
[46] loss: 0.007406
[47] loss: 0.007345
[48] loss: 0.007301
[49] loss: 0.007252
[50] loss: 0.007219
[51] loss

[397] loss: 0.005997
[398] loss: 0.005994
[399] loss: 0.005994
[400] loss: 0.005995
[401] loss: 0.005992
[402] loss: 0.005997
[403] loss: 0.005989
[404] loss: 0.005989
[405] loss: 0.005991
[406] loss: 0.005991
[407] loss: 0.005988
[408] loss: 0.005988
[409] loss: 0.005985
[410] loss: 0.005982
[411] loss: 0.005981
[412] loss: 0.005978
[413] loss: 0.005979
[414] loss: 0.005979
[415] loss: 0.005977
[416] loss: 0.005981
[417] loss: 0.005981
[418] loss: 0.005981
[419] loss: 0.005974
[420] loss: 0.005971
[421] loss: 0.005972
[422] loss: 0.005969
[423] loss: 0.005970
[424] loss: 0.005970
[425] loss: 0.005971
[426] loss: 0.005965
[427] loss: 0.005965
[428] loss: 0.005963
[429] loss: 0.005963
[430] loss: 0.005960
[431] loss: 0.005964
[432] loss: 0.005960
[433] loss: 0.005961
[434] loss: 0.005957
[435] loss: 0.005957
[436] loss: 0.005956
[437] loss: 0.005953
[438] loss: 0.005953
[439] loss: 0.005953
[440] loss: 0.005953
[441] loss: 0.005949
[442] loss: 0.005961
[443] loss: 0.005953
[444] loss: 0

In [44]:
len(training_loss)
validation_loss[0]

0.025526988694048302

In [46]:
trainingLoss= pd.DataFrame({'Loss':training_loss})
validLoss= pd.DataFrame({'Loss':validation_loss})

In [47]:
trainingLoss.to_csv('trainLoss.csv',index=None,sep=' ')
validLoss.to_csv('validLoss.csv',index=None,sep=' ')

In [ ]:
#save the loss
len(training_loss)
lossBatch32= pd.DataFrame({'Loss':training_loss})


In [16]:
#predict the model:
#function for testing the model
def testModel(loader):
    predictedLabel= []
    maxList= []
    trueLable= []
    maxProb= []
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        newMaxProb,maxIndex = torch.topk(output,1,dim=1)
        
        trueLable.append(lable)
        
        if(newMaxProb>0.06):
            predictedLabel.append(1)
        else:
            predictedLabel.append(0)
        
    
    return predictedLabel,trueLable,maxProb


In [ ]:
#save the loss


In [17]:
#test the model on test set
predictedLable,trueLable,maxList= testModel(testLoader)



In [18]:
totalLength= len(trueLable)
TP=FP=FN=TN= 0
for i in range(totalLength):
    if(int(trueLable[i])==1 and predictedLable[i]==1):
        TP += 1
    elif(int(trueLable[i])==1 and predictedLable[i]==0):
        FN += 1
    elif(int(trueLable[i])==0 and predictedLable[i]==0):
        TN += 1
    elif(int(trueLable[i])==0 and predictedLable[i]==1):
        FP += 1

print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)

print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)





TP: 256

FN: 21

FP: 1692

TN: 2657

Result 
Accuracy: 62.970168612191955
Sensitivity: 92.4187725631769
Specificity: 61.09450448378938


In [19]:
#code for the ranking algorithm to test whether the model is correct or not.
def ranking(loader):
    listDic= []        
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        newMaxProb,maxIndex = torch.topk(output,1,dim=1)
        #probabilityDic[output.item()]= lable.item()
        listDic.append((newMaxProb,lable.item()))
    return listDic


In [22]:
probabilityDic= ranking(newTrainLoader)

print("Length of Dictionary:",len(probabilityDic))

#sort the list
probabilityDic.sort()

Length of Dictionary: 8634


In [ ]:
int(probabilityDic[0][1])

In [ ]:
print(newCombineTrain.shape)

In [23]:
#create the 10 buckets of equal size except 1
bucket1=[]
bucket2=[]
bucket3=[]
bucket4=[]
bucket5=[]
bucket6=[]
bucket7=[]
bucket8=[]
bucket9=[]
bucket10=[]

In [24]:
for i in range(len(probabilityDic)):
    if(i<1082):
        bucket1.append(int(probabilityDic[i][1]))
    elif(i<2161):
        bucket2.append(int(probabilityDic[i][1]))
    elif(i<3240):
        bucket3.append(int(probabilityDic[i][1]))
    elif(i<4391):
        bucket4.append(int(probabilityDic[i][1]))
    elif(i<5398):
        bucket5.append(int(probabilityDic[i][1]))
    elif(i<6477):
        bucket6.append(int(probabilityDic[i][1]))
    elif(i<7556):
        bucket7.append(int(probabilityDic[i][1]))
    elif(i<8635):
        bucket8.append(int(probabilityDic[i][1]))
    elif(i<9714):
        bucket9.append(int(probabilityDic[i][1]))
    else:
        bucket10.append(int(probabilityDic[i][1]))

In [36]:
def countOnesInBucket(bucket1,bucket2,bucket3,bucket4,bucket5,
                     bucket6,bucket7,bucket8,bucket9,bucket10):
    countOne= []
    countOne.append(bucket1.count(1))
    countOne.append(bucket2.count(1))
    countOne.append(bucket3.count(1))
    countOne.append(bucket4.count(1))
    countOne.append(bucket5.count(1))
    countOne.append(bucket6.count(1))
    countOne.append(bucket7.count(1))
    countOne.append(bucket8.count(1))
    countOne.append(bucket9.count(1))
    countOne.append(bucket10.count(1))
    
    countOne.sort()
    return countOne

In [37]:
actual= countOnesInBucket(bucket1,bucket2,bucket3,bucket4,bucket5,
                     bucket6,bucket7,bucket8,bucket9,bucket10)

In [38]:
print("Bucket actual:",actual)


Bucket actual: [0, 0, 3, 4, 6, 7, 59, 112, 151, 175]


In [39]:
#make the test set also balanced inorder to calculate the cost
totalFraud= trueLable.count(1)
totalNonFraud= trueLable.count(0)
print("TotalFraud:",totalFraud)
print("TotalNonFraud:",totalNonFraud)

TotalFraud: 277
TotalNonFraud: 4349


In [40]:
#weight for fradulent and non-fraudulent
weightFalseAlaram= totalNonFraud/(totalFraud + totalNonFraud)
weightMisses= (totalFraud)/(totalFraud + totalNonFraud)

print("Weight of FalseAlarm:",weightFalseAlaram)
print("Weight of misses:",weightMisses)


Weight of FalseAlarm: 0.9401210549070471
Weight of misses: 0.05987894509295288


In [ ]:
#under sample the newCostTest


In [41]:
#cost model
#Avg cost per Claim: USD 2640
#Avg Cost per Investigation: $203


totalFraudCase= TP+FN
totalNonFraudCase= TN+FP

#initialize the cost factor
avgCostPerClaim= 2640
avgCostperInvestigation= 203

falseAlarm= weightFalseAlaram * FN * (avgCostPerClaim + avgCostperInvestigation) #costNonFraudMissClassify
misses= weightMisses * FP * avgCostPerClaim    #costFraudMissClassify



totalCost= falseAlarm + misses

print("No of FraudMissClassify:",FN)
print("Number of nonfraudMissClassify:",FP)
print("totalFraud Case:", totalFraudCase)
print("totalNonFraud Case:", totalNonFraudCase)
print("Cost of false alarm:",falseAlarm)
print("Cost of misses:",misses)

print("totalCost of the model:",totalCost)

No of FraudMissClassify: 21
Number of nonfraudMissClassify: 1692
totalFraud Case: 277
totalNonFraud Case: 4349
Cost of false alarm: 56128.04734111543
Cost of misses: 267472.0622568093
totalCost of the model: 323600.10959792475


In [ ]:
#save this model

torch.save(model.state_dict(),'./bestAnnPath')

In [ ]:
#load  the model
loadedModel= Network()

In [ ]:
loadedModel.load_state_dict(torch.load("./bestAnnPath"))